In [2]:
import torch
import torch.nn as nn
from models.dpt import DPTSegmentationWithCarbon
from dataset import CarbonDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from models.util import calculate_correlation
kwargs = {
    "num_classes": 7,
}

model = DPTSegmentationWithCarbon(num_classes=kwargs["num_classes"])
path = "Dataset/Training/image/SN10_Forest_IMAGE"
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
data_loader = DataLoader(CarbonDataset(path,transform, mode = "Train"), batch_size=2, shuffle=True)

x, carbon,gt = data_loader.__iter__().__next__()
gt_pred, carbon_pred = model(x)
corr = calculate_correlation(gt_pred, gt)
print(corr)


c:\ProgramData\anaconda3\envs\dust\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\anaconda3\envs\dust\Lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


Dataset/Training/image/SN10_Forest_IMAGE Done./
Dataset/Training/image/SN10_Forest_SH Done./
Dataset/Training/label/SN10_Forest_Carbon Done./
Dataset/Training/label/SN10_Forest_GT Done./


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
print(model.pretrained)


Module(
  (model): VisionTransformer(
    (patch_embed): HybridEmbed(
      (backbone): ResNetV2(
        (stem): Sequential(
          (conv): StdConv2dSame(4, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
          (norm): GroupNormAct(
            32, 64, eps=1e-05, affine=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
        )
        (stages): Sequential(
          (0): ResNetStage(
            (blocks): Sequential(
              (0): Bottleneck(
                (downsample): DownsampleConv(
                  (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (norm): GroupNormAct(
                    32, 256, eps=1e-05, affine=True
                    (drop): Identity()
                    (act): Identity()
                  )
                )
                (conv1): StdConv2

In [ ]:
print(model.scratch)

Module(
  (layer1_rn): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer2_rn): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer3_rn): Conv2d(768, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer4_rn): Conv2d(768, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (refinenet1): FeatureFusionBlock_custom(
    (out_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (resConfUnit1): ResidualConvUnit_custom(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (skip_add): FloatFunctional(
        (activat

In [4]:
from torchinfo import summary

# 모델과 입력 데이터의 크기를 summary 함수에 전달
summary(model,col_names=("input_size","output_size","num_params",),row_settings=('var_names','depth'), depth=10,input_size=(2, 4, 224, 224))


Layer (type (var_name):depth-idx)                                           Input Shape               Output Shape              Param #
Segformer (Segformer)                                                       [2, 4, 224, 224]          [2, 4, 56, 56]            --
├─MiT (mit): 1-1                                                            [2, 4, 224, 224]          [2, 32, 56, 56]           --
│    └─ModuleList (stages): 2-1                                             --                        --                        --
│    │    └─ModuleList (0): 3-1                                             --                        --                        --
│    │    │    └─Unfold (0): 4-1                                            [2, 4, 224, 224]          [2, 196, 3136]            --
│    │    │    └─Conv2d (1): 4-2                                            [2, 196, 56, 56]          [2, 32, 56, 56]           6,304
│    │    │    └─ModuleList (2): 4-3                                       

In [5]:
from models.segformer_simple import Segformer
args = {
    'dims': (32, 64, 160, 256),
    'heads': (1, 2, 5, 8),
    'ff_expansion': (8, 8, 4, 4),
    'reduction_ratio': (8, 4, 2, 1),
    'num_layers': 2,
    'channels': 4,
    'decoder_dim': 256,
    'num_classes': 4
}
model = Segformer(**args)
summary(model,col_names=("input_size","output_size","num_params",),row_settings=('var_names','depth'), depth=10,input_size=(2, 4, 256, 256))

Layer (type (var_name):depth-idx)                                           Input Shape               Output Shape              Param #
Segformer (Segformer)                                                       [2, 4, 256, 256]          [2, 4, 64, 64]            --
├─MiT (mit): 1-1                                                            [2, 4, 256, 256]          [2, 32, 64, 64]           --
│    └─ModuleList (stages): 2-1                                             --                        --                        --
│    │    └─ModuleList (0): 3-1                                             --                        --                        --
│    │    │    └─Unfold (0): 4-1                                            [2, 4, 256, 256]          [2, 196, 4096]            --
│    │    │    └─Conv2d (1): 4-2                                            [2, 196, 64, 64]          [2, 32, 64, 64]           6,304
│    │    │    └─ModuleList (2): 4-3                                       